# LUX Local analysis notebook

In [ ]:
import warnings
import os
# Define writable cache directories to avoid container errors
# 1. For Matplotlib's own cache
os.environ['MPLCONFIGDIR'] = os.path.join(os.getcwd(), ".matplotlib_cache")
# 2. For the fontconfig library cache
os.environ['XDG_CACHE_HOME'] = os.path.join(os.getcwd(), ".cache")

import plotly.io as pio
pio.renderers.default = 'notebook'
from lux_noctis.lux_qc import LuxFragPipeProtein, CorrelationMatrix
from lux_noctis.goe import Go
from IPython.display import HTML
import re
import json

## User defined parameters

Adjust these parameters to match FragPipe being QCed.

For a basic analysis, these are all only parameters that should be changed in the notebook.

In [ ]:
# The labels for the 2 classes that will be compared for differential abundance
class_label1 = None
class_label2 = None

# The folder with the outpu of the FragPipe analysis
input_folder = '../FragPipe'

# Uniprot annotation for proteins searched by FragPipe.
# Can be generated via lux_noctis.download_uniprot_annotation()
uniprot_annotation_filename = '/mnt/wollscheid/Databases/uniprot_annotation.tsv'

# Surfy annotation file.
# Can by generated via lux_noctis.download_surfy()
surfy_filename = "/mnt/wollscheid/Databases/surfy.txt"

# Quantile threshold for removing proteins with low abundance values
quantile_threshold = 0.25

# Minimum number of peptide a protein has to have to be considered
min_peptide_count = 2

# Should the data be median normalized?
normalize = True

# Should missing values be imputed?
impute = False

# Should the data be filtered based on CVs?
filter_cv = True

# Drop all samples that are not class_label1 or class_label2
reduce_to_labels = False

# List of samples to drop
drop_samples = []

# Links to GO files
gaf_filename = '/mnt/wollscheid/Databases/goa_human.gaf'
obo_filename = '/mnt/wollscheid/Databases/go-basic.obo'

In [ ]:
# If passed by papermill we get a string that needs to be conveted
# to a python list
if isinstance(drop_samples, str):
    drop_samples = json.loads(drop_samples.replace("'", '"'))

# Main class initialization
lux = LuxFragPipeProtein(
    input_folder=input_folder,
    #uniprot_annotation_filename="/mnt/phrt/Webtop/Databases/uniprotkb_AND_reviewed_true_AND_model_o_2023_07_13.tsv",
    uniprot_annotation_filename=uniprot_annotation_filename,
    surfy_filename=surfy_filename,
    quantile_threshold=quantile_threshold,
    min_peptide_count=min_peptide_count,
    reduce_to_labels=reduce_to_labels,
    class_label1=class_label1,
    class_label2=class_label2,
    drop_samples=drop_samples
)

# GO enrichment class initialization
goa = Go(
    gaf_filename=gaf_filename,
    obo_filename=obo_filename
)

## Overview of protein identifications across runs

We start off by looking at the distribution of protein identifications across samples and classes.

### Total protein identifications per sample

**What to look for:**  
- In LuxLocal the bulk of protein identifications is composed of contaminants. 
Therefore we expect little variation across the classes.

In [ ]:
lux.plot_proteins_per_sample()

### Distribution of protein identifications across samples

**NOTA BENE**: only classes with at least 10 protein ids are shown

**What to look for:**  
- In LuxLocal the bulk of protein identifications is composed of contaminants. Therefore we expect little variation across the samples.

In [ ]:
with warnings.catch_warnings():
    # Suppress future warnings for external code
    warnings.simplefilter("ignore", category=FutureWarning)
    lux.plot_upset_proteins(10)

## Overview of protein abundances across runs

This is were start looking at the distributions of protein abundances across classes/samples.

### Unnormalized protein abundances

**What to look for:**  
- In LuxLocal the bulk of protein identifications is composed of contaminants. Therefore we expect similar distributions and equal medians.

In [ ]:
lux.plot_intensity_distributions_across_samples()

### Median normalized protein abundances

If the prior plot show unequal medians we can median normalize protein abundances.

**What to look for:** 

- After median normalization the abundance distributions should be very similar to each other.
- Large divergence in the medians after normalization could indicate that a sample has low intensity (quantile_threshold) or poor coverage (min_peptide_count).

If this is not the case, more complex normalization/batch correction might be required.

In [ ]:
if normalize:
    lux.normalize_protein_abundance_global()
    lux.plot_intensity_distributions_across_samples()

### Imputed

If desired missig values can be imputed.

Note, that a very simple imputation is used. Nans are filled with the minimum intensity measured in the corresponding sample minus a random percentage (between 1 and 10%) of said value.

Note, imputing is probably a bad idea and should not be used without a good reason.

In [ ]:
if impute:
    lux.impute_protein_abundance()
    lux.plot_intensity_distributions_across_samples()

## Coefficient of variation

Next we have a look at the coefficient of variation across the classes.

**What to look for:**  
- In label free quantification we are normally looking for CVs below 20%

### Initial

In [ ]:
lux.plot_cv()

### Filtered

If desired the dataset can be filtered to remove proteins with high CVs.

By default all proteins with CV higher than 20 in all classes are removed.

In [ ]:
if filter_cv:
    lux.filter_by_cv()
    lux.plot_cv()

## Overview of sample similarities

Next we have a look at sample similarities across classes.

### Intensity based clustering

First we test a simple clustering by protein intensity.

**What to look for:**  

- We would like to see samples from the same class clustering together.
- We should also make sure that we don't have too many missing values.

In [ ]:
lux.plot_abundance_clustermap()

### Correlation based clustering

Second we test a correlation based clustering.

**What to look for:**  

- We would like to see samples from the same class clustering together.

In [ ]:
lux.plot_corr_clustermap(figsize=(10,10))

### PCA clustering

Finally we try to cluster the samples by principla component analysis.

**What to look for:**  

- We would like samples from the same class clustering together.

In [ ]:
lux.plot_pca()

## Differential abundance

In this section we compare protein abundances in desired classes.

### Bayes moderated t-test

We implement the bayes moderated t-test workflow with Benjamini-Hochberg correction from limma.

**NOTA BENE**

Even though we are only interested in 2 classes, if additional samples are present, these are kept in the analysis to increase statistical power.  
If this is not desired behaviour, adjust the third parameter (class_labels) of limma_ttest().

In [ ]:
_ = lux.limma_ttest(class_label1, class_label2, lux.experiment_annotation['class'].unique())

Save:
- top-table
- class1 singularities
- peptides associated with significantly up or class1 singularity proteins

to file.

In [ ]:
lux.top_table.to_csv('top-table_{}_vs_{}.csv'.format(class_label1, class_label2), index=False)
lux.singularities_class1.to_csv('singularities_{}.csv'.format(class_label1))
lux.get_significant_peptides(side=1).to_csv('significant_peptides_{}.csv'.format(class_label1))

### Volcano plot

Visualize the result of the previous analysis in a volcano plot.

**What to look for:**  

- Proteins with significantly different abundance across the tested classes.

Do not over-interpret mean(log2(fold change)).

Be very carefull drawing any conclusion based on identifications that don't meet the significance threshold (i.e. remember that we are only testing to reject the null hypothesis)!

In [ ]:
lux.plot_volcano(class_label1, class_label2)

### Membrane enrichment of differentially abundant proteins

We visualize the membrane enrichment in the significantly changing proteins.

**NOTA BENE**  
By default only up-regulated proteins are considered. If this is not desired behaviour adjust the side parameter of get_membrane_enrichment():
- 0 for both
- 1 for up
- -1 for down

**What to look for:**  

- We would like to see that the majority of significantly enriched proteins are membrane proteins

In [ ]:
lux.get_membrane_enrichment(side=1, fc_cutoff=2)

### Intensity distributions of differentially abundant proteins

We plot the intensity of differentially abundant proteins against the overall protein abundance distributions.

**What to look for:**  

- How does the intensity of differentially abundant proteins compare to overall protein abundances? Are they actually enriched in one of the conditions?

NOTE: The legend is sorted by log odds.

In [ ]:
lux.plot_interactive_intensity_of_significantly_changed_proteins(p_cutoff=0.01, fc_cutoff=1)

### Heatmap visualization of differentially abundant proteins

We visualize the intensity of differentially abundant proteins as a heatmap

In [ ]:
lux.plot_clustermap_of_significantly_changed_proteins(p_cutoff=0.01)

## Proteins unique to one class

If a protein is only present in one of the 2 classes, it will be missing in the results of the Bayes moderated t-test. Let's look at them separately.

### Protein abundance distributions

Plot the intensity of proteins unique to one class as a barplot.

**What to look for:**  

- Compare the intensity of  

In [ ]:
lux.plot_interactive_singularities(class_label1, class_label2)

### Membrane enrichment of unique proteins

We visualize the membrane enrichment in the significantly changing proteins.

**NOTA BENE**  
By default only proteins unique to class_label1 are considered. If this is not desired behaviour adjust the cls parameter of get_membrane_enrichment_singularities():
- 1 for class_label1
- 2 for class_label2

**What to look for:**  

- We would like to see that the majority of significantly enriched proteins are membrane proteins

In [ ]:
lux.get_membrane_enrichment_singularities()

## Gene Ontology Enrichment analysis of differentially abundant proteins

Look for GO term enrichment in the differentially abundant and unique proteins

**What to look for:**  

- Terms associated with plasma membrane or the process being investigated.

In [ ]:
goa.run_go_enrichment_on_top_table(lux.top_table, top_table_p_cutoff=0.01, top_table_fc_cutoff=1, 
                                   extra_entries=lux.singularities_class1.columns)

In [ ]:
if goa.go_results:
    sig_go = goa.go_results_df[goa.go_results_df['p_bonferroni'] < 0.05]
    display(sig_go)

## Peptide level

### Summary peptide table

Generate a table summarizing all peptides for the significant and singularity proteins.

In [ ]:
HTML(lux.protter_table())